In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
us_electoral_college_wiki_page_link = 'https://en.wikipedia.org/wiki/United_States_Electoral_College'
request = requests.get(us_electoral_college_wiki_page_link)
page = BeautifulSoup(request.text)

In [3]:
table = page.find(lambda tag: tag.name == 'big' and 'presidential electors' in tag.text).find_parent('table', class_='wikitable sortable')

In [4]:
years_headers = table.find_all(lambda tag: tag.name == 'th' and bool(tag.find('a')) and '\'' in tag.find('a').text)

In [5]:
first_state_row = table.find(lambda tag: tag.name == 'a' and tag.attrs['href'] == '/wiki/Alabama').find_parent('tr')

In [6]:
state_rows = [first_state_row] + first_state_row.find_next_siblings(lambda tag: tag.name == 'tr' and bool(tag.find('a')))

In [7]:
def map_links_to_election_years(links):
    return list(map(lambda x: int(x.attrs['title'][:4]), links))

In [8]:
years_headers = list(map(lambda x: map_links_to_election_years(x.find_all('a')), years_headers))

In [9]:
electoral_college_vote_count_col = []
year_col = []
state_col = []

In [10]:
for state in state_rows:
    state_name = state.select('th a')[0].text
    count_numbers = state.find_all('td')
    
    for index, count_ele in enumerate(count_numbers):
        years = years_headers[index]
        year_count = len(years)
        vote_count = int(count_ele.text.strip() or 0)
    
        year_col += years
        electoral_college_vote_count_col += ([vote_count] * year_count)
        state_col += ([state_name] * year_count)

In [11]:
df = pd.DataFrame({
    'Year': year_col,
    'Electoral college vote count': electoral_college_vote_count_col,
    'State': state_col
})

In [12]:
df.to_csv('../data/electoral_college_vote.csv', index=False)

In [22]:
df.sample(20)

,Year,Electoral college vote count,State
1860,1912,3,New Mexico
219,1956,8,Arkansas
224,1976,6,Arkansas
2306,1808,4,Rhode Island
1828,2020,14,New Jersey
274,1940,22,California
1669,1856,0,Nevada
473,1792,3,Delaware
752,1964,4,Idaho
330,1928,6,Colorado
